In [ ]:
import numpy as np
import pandas as pd
import re # Used for the conversion of "r##_c##" in only the numbers --> TODO: check where it comes from
from helpers import *
from play_with_data import *
from pre_processing import *
from matrix_factorization import *
from cross_validation import *


# Done before the exercise

In [ ]:
#********* Load the given data **********
r_c, x = load_data_old('./data_train.csv')

In [ ]:
#********* Creation of a matrix of the data ********
nUser = 10000
nItem = 1000
data = np.zeros([nUser, nItem]) # These numbers were given

for ind, i in enumerate(r_c): # Loop over all the ID 
    data[int(re.findall('\d+', i)[0])-1, int(re.findall('\d+', i)[1])-1] = x[ind] # Use the information in the ID (row, col) to create the matrix

print(data[:10,:10])


In [ ]:
print(r_c[:10],'\n',x[:10])

In [ ]:
df = pd.DataFrame(data)
a = []
for i in range(df.shape[0]):
    a.append(len(np.where(df.loc[i] != 0)[0]))
#print(a)
#print(len(a))
print(np.std(a))
print(np.mean(a))
#display(df)

In [ ]:
#********** Data preview ************
# Check if there is any missed data 
# It was told us that we have the data from 10'000 users for 1000 films, but we don't have all these data
info_general(nUser, nItem, x, data)
print('\n')
info_ratings(data)


With this barplot, we can see that ratings are not distributed in an uniform way, this may suggest that there is a bias in the rating matrix that has to be considered.

# Done after the exercise

In [ ]:
#******** Creation of a sparse matrix of the data **********
ratings = load_data('./data_train.csv') 
print(ratings.shape) # rows are "users" and "items" are the columns

In [ ]:
#******* Create DataFrame with the good representation of the data **********
users_ID = np.arange(ratings.shape[0]) # To start from 0 and finish at 999 = 1'000 indices
movies_ID = np.arange(ratings.shape[1]) # To start from 0 and finish at 9999 = 10'000 indices

users_ID_rep = users_ID.repeat(ratings.shape[1]) # To do the repeat like that: 0 0 0 0 ... 1 1 1  .... 999 ... 999
movies_ID_rep = np.tile(movies_ID, ratings.shape[0]) # To do the repeat like that: 0 1 2 3 4...9999 0 1 2 3 ... 9999

all_ratings = ratings.todense() # In order to be able to handle this like numpy
all_ratings = np.ravel(all_ratings) # To put in one dimension

# Creation of a dictionary to create the dataframe in the correct representation
IDs_dict = {'movies ID': movies_ID_rep,
            'ratings': all_ratings,
            'users ID': users_ID_rep
           }

ratings_representation = pd.DataFrame.from_dict(IDs_dict) # Creation of the dataframe from the dictionary

dataF = ratings_representation[ratings_representation.ratings != 0]
print(dataF)


In [ ]:
#***** Data preview *********
num_items_per_user, num_users_per_item = plot_raw_data(ratings) # Original code is from the course, ex10 'plots.py'
print("Maximum number of items per user:\t{}\nMinimum number of items per user:\t{}\n".format(np.max(num_items_per_user), np.min(num_items_per_user)))
print("Maximum number of users per item:\t{}\nMinimum number of users per item:\t{}".format(np.max(num_users_per_item), np.min(num_users_per_item)))

### Question

   - Il est normal de mettre un threshold (min_num_ratings) parce qu'on veut un certain nombre de données pour ce prononcer quand à donner une note, ceci implique qu'on enlève des users et des items, du coup notre matrice des ratings va être plus petite. Quand on la remplis et on fait un submit, comment est-ce qu'on gére ça?
       - Ici je suppose que c'est bien de faire la selection des ratings pour le train et test --> comme ceci on est pas ou moins biaisé par les movies et users qui n'ont que des 0 et qui donc n'apporte rien apart du "bruit"
       - Pour le remplissage de ce que l'on doit submit j'ai fait une petite comparaison entre les data que l'on nous donne et ce qui se trouve dans le sample_submission, (enfin je vais le faire) 

In [ ]:
#********** Pre-processing **********
min_num_ratings = 10 # or 15 this is based on the information given above
ratings_valid = valid_ratings(ratings, num_items_per_user, num_users_per_item,min_num_ratings)
train, test = split_data(ratings_valid) # This will put 90% of the items for the users that have at least one non-zero entry and the 10% in test


In [ ]:
cross_validation_application(train ,False ,4,0.01, 20, 0.2, 0.3,1)

In [ ]:
#********* Try ************
#rmse = rmse_movie_mean(train, test)
rmse = matrix_factorization_SGD(train, test)

In [ ]:
rmse

# Done so far
- Preview of the data
- Pre-processing:
    - Choosing only the "valid ratings", the users and items that contains more than min_num_ratings
    - Splitting the data in test and train, by choosing 90% of the ratings from the valid_ratings and only the non-zeros values

In [ ]:
from surprise import SVD
from surprise import dataset
from surprise import Dataset
from surprise import evaluate, print_perf

reader = dataset.Reader(rating_scale=(1, 5))



ratings_surpr=Dataset.load_from_df(dataF[['users ID', 'movies ID', 'ratings']], reader)

ratings_surpr.split(n_folds=3)




In [ ]:


# We'll use the famous SVD algorithm.
algo = SVD()

# Evaluate performances of our algorithm on the dataset.
perf = evaluate(algo, ratings_surpr, measures=['RMSE'])

print_perf(perf)
